binanry decisionn variable sp (eg berkely admissions)
2 grouping varibles & a binary
 - 1 grouping variable eg gender
 - grouping variable has class imbalance
mixed: use continus variables to define the decsion variable.


## To induce SP in rates

we need the grouping variable to have onen rate of success in decision variable that is the opposite rate of the overall, so we need to sam

In [19]:
import numpy as np
import pandas as pd
import string
import matplotlib.pylab as plt

The causal explanation for rate-based SP is that the application rates by gender varied in a way that was correlated with the department size and/or acceptance rate.  Here we create a model that will, for most samples, have SP for some departments. We only set the portion of applications to each department, the rate of each gender applying to each department and each department's acceptance rate.  

In [25]:
p_dept = [.15,.15,.1,.6]
p_gender_dept = [[.7, .3],[.8,.2],[.85,.15],[.2,.8]]
gender_list = ['F','M']
p_admit_dept = [[.15,.85], [.18,.82], [.25,.75],[.3,.7]]
# need to have higher accept in the larger subgroup, larger subgroup should have opposite protected class balance
N = 1000

d = np.random.choice(list(range(len(p_dept))), size=N, p =p_dept)
g = [np.random.choice(gender_list, p =p_gender_dept[d_i]) for d_i in d]
a = [np.random.choice([1,0],p = p_admit_dept[d_i]) for d_i in d]
data = [[d_i,g_i,a_i] for d_i, g_i,a_i in zip(d,g,a)]

df = pd.DataFrame(data = data, columns=['department','gender','decision'])
df.head()

,department,gender,decision
0,0,M,0
1,3,M,1
2,0,F,0
3,3,M,1
4,1,F,0


We can check that the probabilities match what we set.  First the per-department admission rate.

In [50]:
actual_admit_dept = df.groupby(['department']).mean()
expected_admit_dept = [p[0] for p in p_admit_dept]
actual_admit_dept['expected'] = expected_admit_dept
actual_admit_dept

,decision,expected
department,,
0,0.106918,0.15
1,0.197279,0.18
2,0.244444,0.25
3,0.293046,0.30


In [60]:
actual_app_dept = df.groupby(['department'])['gender'].value_counts().unstack()
actual_app_dept

gender,F,M
department,,
0,112,47
1,121,26
2,79,11
3,118,486


In [64]:
expected_app_dept_dat = [[p_d[0]*n_d, p_d[1]*n_d] for p_d,n_d in zip(p_gender_dept,[p*N for p in p_dept])]
expected_app_dept = pd.DataFrame(data = expected_app_dept_dat, columns = gender_list)
expected_app_dept

,F,M
0,105.0,45.0
1,120.0,30.0
2,85.0,15.0
3,120.0,480.0


Next we can look for SP

In [77]:
df.groupby('gender')['decision'].mean()

gender
F    0.238839
M    0.262681
Name: decision, dtype: float64

In [78]:
df.groupby(['gender','department']).mean().unstack()

decision          
department         0         1
gender                        
F           0.244838  0.220183
M           0.273902  0.236364

To detect SP, we compare which group is the highest in the overall to to which group is highest in each of the departments. 

In [79]:
# now detect and compare which trends
overall = df.groupby('gender')['decision'].mean().idxmax()
print('overall more admitted: ', overall)

per_dept = [gender_list[g] for g in np.argmax(df.groupby(['gender','department']).mean().unstack().values,axis=0)]
print('per dept more addmitted:', per_dept)
# it's sp for each dept that is not the same as the overall
[not(dept==overall) for dept in per_dept]

overall more admitted:  M
per dept more addmitted: ['M', 'M']


[False, False]

Now, we can run it a bunch more times and count how many SP in each trial, to see how reliable it is.

In [80]:
sp_occ = []

for i in range(20):
    p_dept = [.15,.15,.1,.6]
    p_gender_dept = [[.7, .3],[.8,.2],[.85,.15],[.2,.8]]
    gender_list = ['F','M']
    p_admit_dept = [[.15,.85], [.18,.82], [.25,.75],[.3,.7]]
    # need to have higher accept in the larger subgroup, larger subgroup should have opposite protected class balance
    N = 1000

    d = np.random.choice(list(range(len(p_dept))), size=N, p =p_dept)
    g = [np.random.choice(gender_list, p =p_gender_dept[d_i]) for d_i in d]
    a = [np.random.choice([1,0],p = p_admit_dept[d_i]) for d_i in d]
    data = [[d_i,g_i,a_i] for d_i, g_i,a_i in zip(d,g,a)]

    df = pd.DataFrame(data = data, columns=['department','gender','decision'])

    overall = df.groupby('gender')['decision'].mean().idxmax()
    per_dept = [gender_list[g] for g in np.argmax(df.groupby(['gender','department']).mean().unstack().values,axis=0)]


    sp_occ.append(sum([not(dept==overall) for dept in per_dept]))
    
sp_occ

[2, 0, 0, 3, 3, 2, 2, 2, 0, 2, 1, 2, 0, 2, 2, 2, 1, 1, 1, 1]

We see from above that it generates SP most of the time but a varialbe number of times for the exact same settings to this sampling method does not reliably induce SP, it does show that, with no malintent that SP can occur.  

# intentional SP

To more reliably induce rate based SP we should sample less independently.  


In [88]:
p_dept = [.15,.15,.1,.6]
p_gender_dept = [[.7, .3],[.8,.2],[.85,.15],[.2,.8]]
gender_list = ['F','M']
p_admit_dept_gender = [{'F':.15,'M':.1},{'F':.17,'M':.06},
                       {'F':.30,'M':.24},{'F':.35,'M':.30}] 
# need to have higher accept in the larger subgroup, larger subgroup should have opposite protected class balance
N = 1000

d = np.random.choice(list(range(len(p_dept))), size=N, p =p_dept)
g = [np.random.choice(gender_list, p =p_gender_dept[d_i]) for d_i in d]
p_admit =[ p_admit_dept_gender[d_i][g_i] for d_i,g_i in zip(d,g)]
p_admit
a = [np.random.choice([1,0], p = [p,1-p]) for p in p_admit]
data = [[d_i,g_i,a_i] for d_i, g_i,a_i in zip(d,g,a)]

df = pd.DataFrame(data = data, columns=['department','gender','decision'])
df.head()

,department,gender,decision
0,0,F,0
1,3,M,0
2,3,M,0
3,0,M,0
4,3,M,0


In [89]:
p_admit_dept_gender[1]['M']

0.06

In [90]:
df.groupby('gender')['decision'].mean()

gender
F    0.221445
M    0.246935
Name: decision, dtype: float64

In [91]:
df.groupby(['gender','department']).mean().unstack()

decision                              
department         0         1         2         3
gender                                            
F           0.150538  0.164062  0.220930  0.336066
M           0.040816  0.088235  0.166667  0.281513

In [92]:
overall = df.groupby('gender')['decision'].mean().idxmax()

per_dept = [gender_list[g] for g in np.argmax(df.groupby(['gender','department']).mean().unstack().values,axis=0)]


[not(dept==overall) for dept in per_dept]

[True, True, True, True]

We can try to use 3 categories and do an ordering swap detection, maybe?

In [16]:

p_race_dept = [[.8,.1,.1], [.7,.13,.17],[.5,.2,.3],[.85,.07,.08]]
race_list = ['W','B','H']

r = [np.random.choice(race_list, p =p_race_dept[d_i]) for d_i in d]
df['race'] = r

In [17]:
df.groupby('race')['decision'].mean()

race
B    0.184466
H    0.228346
W    0.266234
Name: decision, dtype: float64

In [18]:
df.groupby(['race','department']).mean().unstack()

decision                              
department         0         1         2         3
race                                              
B           0.190476  0.052632  0.192308  0.243243
H           0.090909  0.200000  0.142857  0.320755
W           0.183333  0.181034  0.302326  0.303462